In [1]:
import json
import time
from flask import Flask, jsonify, request, Response
import requests
import sys
from datetime import datetime

def post_json_request(url, obj):
    return requests.post(url, json=obj).json()

In [7]:
pubmedResult = post_json_request(
    "http://localhost:5000/query",
    {
        "database": "adc_cali",
        "maxdocs": 20,
        "project": "breast_cancer",
        "patternid": 1,
        "query": "breast[Title/Abstract] AND carcinoma[Title/Abstract]",
    },
)[0]["exit"]

In [6]:
print(pubmedResult)

0


In [16]:
documents = post_json_request(
    "http://localhost:5001/mongo-doc-list",
    {
        "db_name": "adc_cali",
        "coll_name": "breast_cancer_metadata_global"
    },
)
documents[0].keys()

dict_keys(['_id', 'title', 'abstract', 'authors', 'dbid', 'doi', 'lang', 'org', 'pat_id', 'url', 'year'])

In [18]:
urlList=list(map(lambda document: {"url": document["url"]}, documents))
urlList

[{'url': ''},
 {'url': ''},
 {'url': ''},
 {'url': ''},
 {'url': ''},
 {'url': ''},
 {'url': ''},
 {'url': ''},
 {'url': ''},
 {'url': 'http://europepmc.org/backend/ptpmcrender.fcgi?accid=PMC9427118&blobtype=pdf'},
 {'url': 'http://europepmc.org/backend/ptpmcrender.fcgi?accid=PMC9424548&blobtype=pdf'},
 {'url': 'http://europepmc.org/backend/ptpmcrender.fcgi?accid=PMC9425891&blobtype=pdf'},
 {'url': 'http://europepmc.org/backend/ptpmcrender.fcgi?accid=PMC9424033&blobtype=pdf'},
 {'url': 'http://europepmc.org/backend/ptpmcrender.fcgi?accid=PMC9424343&blobtype=pdf'},
 {'url': ''},
 {'url': ''},
 {'url': ''},
 {'url': ''},
 {'url': 'http://europepmc.org/backend/ptpmcrender.fcgi?accid=PMC9400791&blobtype=pdf'},
 {'url': 'http://europepmc.org/backend/ptpmcrender.fcgi?accid=PMC9422358&blobtype=pdf'}]

In [33]:
def url2head(url):
    try:
        return post_json_request(
            "http://localhost:5002/url2htext",
            { "url": url }
        )
    except:
        return {"htext": ""}
url2head("http://europepmc.org/backend/ptpmcrender.fcgi?accid=PMC9427118&blobtype=pdf")

{'htext': 'Breast Cancer: Targets and Therapy\n\nDovepress\nopen access to scientific and medical research\n\nOpen Access Full Text Article\n\nREVIEW\n\nA Scoping Review on the Status of Female Breast\nCancer in Asia with a Special Focus on Nepal\nRojana Dhakal 1,2 , Maria Noula 1 , Zoe Roupa 1 , Edna N Yamasaki 1\n1\nDepartment of Life and Health Sciences, University of Nicosia, Nicosia, Cyprus; 2Department of Nursing, School of Health and Allied Sciences,\nPokhara University, Kaski, Gandaki Province, Nepal\n\nCorrespondence: Rojana Dhakal, Department of Nursing, School of Health and Allied Sciences, Pokhara University, Kaski, Gandaki Province, Nepal,\nEmail rojanabuddhi2@gmail.com; dhakal.r@live.unic.ac.cy\n\n'}

In [34]:
urls = list(filter(lambda url: url["url"] != "", urlList))
urls

[{'url': 'http://europepmc.org/backend/ptpmcrender.fcgi?accid=PMC9427118&blobtype=pdf'},
 {'url': 'http://europepmc.org/backend/ptpmcrender.fcgi?accid=PMC9424548&blobtype=pdf'},
 {'url': 'http://europepmc.org/backend/ptpmcrender.fcgi?accid=PMC9425891&blobtype=pdf'},
 {'url': 'http://europepmc.org/backend/ptpmcrender.fcgi?accid=PMC9424033&blobtype=pdf'},
 {'url': 'http://europepmc.org/backend/ptpmcrender.fcgi?accid=PMC9424343&blobtype=pdf'},
 {'url': 'http://europepmc.org/backend/ptpmcrender.fcgi?accid=PMC9400791&blobtype=pdf'},
 {'url': 'http://europepmc.org/backend/ptpmcrender.fcgi?accid=PMC9422358&blobtype=pdf'}]

In [35]:
headers = list(map(lambda url: url2head(url["url"])["htext"], urls))

In [36]:
headers

['Breast Cancer: Targets and Therapy\n\nDovepress\nopen access to scientific and medical research\n\nOpen Access Full Text Article\n\nREVIEW\n\nA Scoping Review on the Status of Female Breast\nCancer in Asia with a Special Focus on Nepal\nRojana Dhakal 1,2 , Maria Noula 1 , Zoe Roupa 1 , Edna N Yamasaki 1\n1\nDepartment of Life and Health Sciences, University of Nicosia, Nicosia, Cyprus; 2Department of Nursing, School of Health and Allied Sciences,\nPokhara University, Kaski, Gandaki Province, Nepal\n\nCorrespondence: Rojana Dhakal, Department of Nursing, School of Health and Allied Sciences, Pokhara University, Kaski, Gandaki Province, Nepal,\nEmail rojanabuddhi2@gmail.com; dhakal.r@live.unic.ac.cy\n\n',
 '',
 '',
 '',
 '',
 'Exploration of Targeted Anti-tumor Therapy\nOpen Access Review\n\nAn overview of the anti-cancer actions of Tanshinones, derived\nfrom Salvia miltiorrhiza (Danshen)\nIrum Naz1, Myriam Merarchi2, Shanaya Ramchandani3, Muhammad Rashid Khan4*, Muhammad Nouman\nMalik

In [38]:
headersFiltered = list(filter(lambda head: head != "", headers))
headersFiltered

['Breast Cancer: Targets and Therapy\n\nDovepress\nopen access to scientific and medical research\n\nOpen Access Full Text Article\n\nREVIEW\n\nA Scoping Review on the Status of Female Breast\nCancer in Asia with a Special Focus on Nepal\nRojana Dhakal 1,2 , Maria Noula 1 , Zoe Roupa 1 , Edna N Yamasaki 1\n1\nDepartment of Life and Health Sciences, University of Nicosia, Nicosia, Cyprus; 2Department of Nursing, School of Health and Allied Sciences,\nPokhara University, Kaski, Gandaki Province, Nepal\n\nCorrespondence: Rojana Dhakal, Department of Nursing, School of Health and Allied Sciences, Pokhara University, Kaski, Gandaki Province, Nepal,\nEmail rojanabuddhi2@gmail.com; dhakal.r@live.unic.ac.cy\n\n',
 'Exploration of Targeted Anti-tumor Therapy\nOpen Access Review\n\nAn overview of the anti-cancer actions of Tanshinones, derived\nfrom Salvia miltiorrhiza (Danshen)\nIrum Naz1, Myriam Merarchi2, Shanaya Ramchandani3, Muhammad Rashid Khan4*, Muhammad Nouman\nMalik1, Sumaira Sarwar1, 

In [37]:
def text2locations(text):
    try:
        return post_json_request(
            "http://localhost:5002/text2locations",
            { "text": text }
        )
    except:
        return {"locations": []}
text2locations("Exploration of Targeted Anti-tumor Therapy\nOpen Access Review\n\nAn overview of the anti-cancer actions of Tanshinones, derived\nfrom Salvia miltiorrhiza (Danshen)\nIrum Naz1, Myriam Merarchi2, Shanaya Ramchandani3, Muhammad Rashid Khan4*, Muhammad Nouman\nMalik1, Sumaira Sarwar1, Acharan S Narula5, Kwang Seok Ahn6*\nDepartment of Biochemistry, Faculty of Biological Sciences, Quaid-i-Azam University, Islamabad 45320, Pakistan\n\n1\n\nFaculty of Pharmacy, University of Paris Descartes, 75006 Paris, France\n\n2\n\nDepartment of Pharmacology-Biomedicine, The University of Melbourne, Parkville, VIC 3010, Australia\n\n3\n\nHigher Education Commission of Pakistan, Islamabad 44000, Pakistan\n\n4\n\nNarula Research, Chapel Hill, NC 27516, USA\n\n5\n\nDepartment of Science in Korean Medicine, College of Korean Medicine, Kyung Hee University, 24 Kyungheedae-ro,\nDongdaemun-gu, Seoul 02447, South Korea\n6\n\n*Correspondence: Muhammad Rashid Khan, Department of Biochemistry, Faculty of Biological Sciences, Quaid-i-Azam\nUniversity, Islamabad 45320, Pakistan. mrkhanqau@yahoo.com; Kwang Seok Ahn, Department of Science in Korean Medicine,\nCollege of Korean Medicine, Kyung Hee University, 24 Kyungheedae-ro, Dongdaemun-gu, Seoul 02447, South Korea. ksahn@\nkhu.ac.kr\nAcademic Editor: Gautam Sethi, National University of Singapore, Singapore\nReceived: April 16, 2020 Accepted: May 17, 2020 Published: June 29, 2020\n\nCite this article: Naz I, Merarchi M, Ramchandani S, Khan MR, Malik MN, Sarwar S, et al. An overview of the anti-cancer\nactions of Tanshinones, derived from Salvia miltiorrhiza (Danshen). Explor Target Antitumor Ther. 2020;1:153-70. https://doi.\norg/10.37349/etat.2020.00010\n\n")

{'locations': ['Tanshinones',
  'Salvia',
  'Biochemistry',
  'Pakistan',
  'Pharmacy',
  'Paris Descartes',
  'Paris',
  'France',
  'Melbourne',
  'Parkville',
  'Australia',
  'Pakistan Pakistan',
  'Science',
  'Korean Medicine',
  'Korean Medicine Seoul',
  'South',
  'Biochemistry Pakistan',
  'Science Korean Medicine',
  'Korean Medicine Seoul South Korea',
  'Singapore',
  'Tanshinones Salvia']}

In [40]:
locations = list(map(lambda text: text2locations(text)["locations"], headersFiltered))
locations

[['Asia',
  'Life',
  'Nicosia',
  'Nicosia Cyprus',
  'Nursing',
  'Health',
  'Nursing Health'],
 ['Tanshinones',
  'Salvia',
  'Biochemistry',
  'Pakistan',
  'Pharmacy',
  'Paris Descartes',
  'Paris',
  'France',
  'Melbourne',
  'Parkville',
  'Australia',
  'Pakistan Pakistan',
  'Science',
  'Korean Medicine',
  'Korean Medicine Seoul',
  'South',
  'Biochemistry Pakistan',
  'Science Korean Medicine',
  'Korean Medicine Seoul South Korea',
  'Singapore',
  'Tanshinones Salvia']]

In [49]:
def text2places(text):
    try:
        return post_json_request(
            "http://localhost:5002/text2places",
            { "text": text }
        )
    except:
        return {"places": []}
text2places("Exploration of Targeted Anti-tumor Therapy\nOpen Access Review\n\nAn overview of the anti-cancer actions of Tanshinones, derived\nfrom Salvia miltiorrhiza (Danshen)\nIrum Naz1, Myriam Merarchi2, Shanaya Ramchandani3, Muhammad Rashid Khan4*, Muhammad Nouman\nMalik1, Sumaira Sarwar1, Acharan S Narula5, Kwang Seok Ahn6*\nDepartment of Biochemistry, Faculty of Biological Sciences, Quaid-i-Azam University, Islamabad 45320, Pakistan\n\n1\n\nFaculty of Pharmacy, University of Paris Descartes, 75006 Paris, France\n\n2\n\nDepartment of Pharmacology-Biomedicine, The University of Melbourne, Parkville, VIC 3010, Australia\n\n3\n\nHigher Education Commission of Pakistan, Islamabad 44000, Pakistan\n\n4\n\nNarula Research, Chapel Hill, NC 27516, USA\n\n5\n\nDepartment of Science in Korean Medicine, College of Korean Medicine, Kyung Hee University, 24 Kyungheedae-ro,\nDongdaemun-gu, Seoul 02447, South Korea\n6\n\n*Correspondence: Muhammad Rashid Khan, Department of Biochemistry, Faculty of Biological Sciences, Quaid-i-Azam\nUniversity, Islamabad 45320, Pakistan. mrkhanqau@yahoo.com; Kwang Seok Ahn, Department of Science in Korean Medicine,\nCollege of Korean Medicine, Kyung Hee University, 24 Kyungheedae-ro, Dongdaemun-gu, Seoul 02447, South Korea. ksahn@\nkhu.ac.kr\nAcademic Editor: Gautam Sethi, National University of Singapore, Singapore\nReceived: April 16, 2020 Accepted: May 17, 2020 Published: June 29, 2020\n\nCite this article: Naz I, Merarchi M, Ramchandani S, Khan MR, Malik MN, Sarwar S, et al. An overview of the anti-cancer\nactions of Tanshinones, derived from Salvia miltiorrhiza (Danshen). Explor Target Antitumor Ther. 2020;1:153-70. https://doi.\norg/10.37349/etat.2020.00010\n\n")

{'places': ['South Korea',
  'Pakistan',
  'France',
  'United States of America',
  'Australia',
  'Singapore',
  'Canada',
  'India',
  'Sweden',
  'New Zealand',
  'Denmark',
  'Indonesia']}

In [51]:
places = list(map(lambda text: text2places(text)["places"], headersFiltered))
places

[['Nepal', 'Cyprus', 'United States of America', 'Italy', 'Philippines'],
 ['South Korea',
  'Pakistan',
  'France',
  'United States of America',
  'Australia',
  'Singapore',
  'Canada',
  'India',
  'Sweden',
  'New Zealand',
  'Denmark',
  'Indonesia']]

In [53]:
def text2emails(text):
    try:
        return post_json_request(
            "http://localhost:5002/text2emails",
            { "text": text }
        )
    except:
        return {"emails": []}
text2emails("Exploration of Targeted Anti-tumor Therapy\nOpen Access Review\n\nAn overview of the anti-cancer actions of Tanshinones, derived\nfrom Salvia miltiorrhiza (Danshen)\nIrum Naz1, Myriam Merarchi2, Shanaya Ramchandani3, Muhammad Rashid Khan4*, Muhammad Nouman\nMalik1, Sumaira Sarwar1, Acharan S Narula5, Kwang Seok Ahn6*\nDepartment of Biochemistry, Faculty of Biological Sciences, Quaid-i-Azam University, Islamabad 45320, Pakistan\n\n1\n\nFaculty of Pharmacy, University of Paris Descartes, 75006 Paris, France\n\n2\n\nDepartment of Pharmacology-Biomedicine, The University of Melbourne, Parkville, VIC 3010, Australia\n\n3\n\nHigher Education Commission of Pakistan, Islamabad 44000, Pakistan\n\n4\n\nNarula Research, Chapel Hill, NC 27516, USA\n\n5\n\nDepartment of Science in Korean Medicine, College of Korean Medicine, Kyung Hee University, 24 Kyungheedae-ro,\nDongdaemun-gu, Seoul 02447, South Korea\n6\n\n*Correspondence: Muhammad Rashid Khan, Department of Biochemistry, Faculty of Biological Sciences, Quaid-i-Azam\nUniversity, Islamabad 45320, Pakistan. mrkhanqau@yahoo.com; Kwang Seok Ahn, Department of Science in Korean Medicine,\nCollege of Korean Medicine, Kyung Hee University, 24 Kyungheedae-ro, Dongdaemun-gu, Seoul 02447, South Korea. ksahn@\nkhu.ac.kr\nAcademic Editor: Gautam Sethi, National University of Singapore, Singapore\nReceived: April 16, 2020 Accepted: May 17, 2020 Published: June 29, 2020\n\nCite this article: Naz I, Merarchi M, Ramchandani S, Khan MR, Malik MN, Sarwar S, et al. An overview of the anti-cancer\nactions of Tanshinones, derived from Salvia miltiorrhiza (Danshen). Explor Target Antitumor Ther. 2020;1:153-70. https://doi.\norg/10.37349/etat.2020.00010\n\n")

{'emails': ['mrkhanqau@yahoo.com']}

In [54]:
emails = list(map(lambda text: text2emails(text)["emails"], headersFiltered))
emails

[['rojanabuddhi2@gmail.com', 'dhakal.r@live.unic.ac.cy'],
 ['mrkhanqau@yahoo.com']]

In [59]:
# pip3 install python-whois
import whois

def is_registered(domain_name):
    try:
        w = whois.whois(domain_name)
    except Exception:
        return False
    else:
        return bool(w.domain_name)

domain_name = "univalle.edu.co"
if is_registered(domain_name):
    whois_info = whois.whois(domain_name)
    print(whois_info)

{
  "domain_name": "univalle.edu.co",
  "registrar": ".CO Internet S.A.S.",
  "whois_server": null,
  "referral_url": null,
  "updated_date": "2022-05-26 17:01:34",
  "creation_date": "1998-07-16 00:00:00",
  "expiration_date": "2022-12-31 23:59:59",
  "name_servers": [
    "dnsase01.univalle.edu.co",
    "dnsase02.univalle.edu.co"
  ],
  "status": "ok https://icann.org/epp#ok",
  "emails": "soporte@cointernet.com.co",
  "dnssec": "unsigned",
  "name": "REDACTED FOR PRIVACY",
  "org": "Universidad el Valle",
  "address": "REDACTED FOR PRIVACY",
  "city": "REDACTED FOR PRIVACY",
  "state": "Valle del Cauca",
  "registrant_postal_code": "REDACTED FOR PRIVACY",
  "country": "CO"
}
